In [9]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

import configparser
import os

ModuleNotFoundError: No module named 'snowflake.snowpark.types.StructType'; 'snowflake.snowpark.types' is not a package

In [2]:
def get_snowpark_session():
    """"Returns an snowpark session object"""

    ##Get configs
    config = configparser.ConfigParser()
    conf_path = os.path.join('C:\\Users\\Esli\\.snowsql','config')
    config.read(conf_path)

    #Snowflake config
    sfAccount = config['connections.dev']['accountname']
    sfUser = config['connections.dev']['username']
    sfPass = config['connections.dev']['password']
    SfRole = config['connections.dev']['rolename']
    sfWarehouse = config['connections.dev']['warehousename']
    sfDatabase = config['connections.dev']['dbname']
    CONNECTION_PARAMETERS = {
        "account": sfAccount,
        "user": sfUser,
        "password": sfPass,
        "database": sfDatabase,
        "warehouse": sfWarehouse,
        "role": SfRole
    }

    return Session.builder.configs(CONNECTION_PARAMETERS).create()


In [7]:
def set_schema(schema_name: str):
    """Set working schema on Snowflake"""
    return session.sql(f'USE SCHEMA {schema_name}').collect()

In [12]:
def read_csv_from_stage(stage_name: str, csv_schema: StructType, csv_pattern: str=None):
    """
    Read the CSV's from given stage and returns as DF, if csv_pattern is not given will read all CSV's from stage.

    Params:
    stage_name (string): The name of the Stage where the files are on snowflake
    csv_schema (StructType): The schema of the file that will be read
    csv_pattern (String) [optional]: The regex expression that will match the files on Stage

    code snippet:
    df = read_csv_from_stage(stage_name = 'RAW_DATA_STAGE', csv_schema=csv_schema, csv_pattern='.*yds_data*[.]csv')    
    """    

    if csv_pattern is not None:
        stage_files = session.sql(f"LIST @{stage_name} PATTERN='{csv_pattern}'")
    else:
        stage_files = session.sql(f"LIST @{stage_name}")

    for row in stage_files.collect():
        csv_path = '@'+row["name"]
        df_csv = session.read.schema(csv_schema).option("skip_header", 1).csv(csv_path)     
        df_csv = df_csv.withColumn('path', F.lit(row["name"]))
        df_csv = df_csv.unionByName(df_csv)

    return df_csv

In [2]:
print('''####################################################
Available packages:
    from snowflake.snowpark.session import Session
    from snowflake.snowpark import functions as F
    from snowflake.snowpark.types import *

    import configparser
    import os

####################################################

Available functions, for details use help(<function_name>:
    get_snowpark_session
    set_schema
    read_csv_from_stage
''')

####################################################
Available packages:
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import udtf

import configparser
import os

####################################################

Available functions:
get_snowpark_session

